In [ ]:
!wget https://github.com/SVizor42/ML_Zoomcamp/releases/download/bee-wasp-data/data.zip

In [ ]:
!unzip data.zip

In [ ]:
# Importing necessary libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img

import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
# Ensure we are using tensorflow v2
print(tf.__version__)

In [ ]:
# Initialize the model
model = Sequential()

# Add a convolutional layer
# 32 filters, kernel size of (3,3), 'relu' activation, input shape of (150, 150, 3)
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))

# Add a max pooling layer
# Pooling size of (2,2)
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten the tensor output from the previous layer
model.add(Flatten())

# Add a dense layer
# 64 neurons, 'relu' activation
model.add(Dense(64, activation='relu'))

# Add the output layer
# 1 neuron, 'sigmoid' activation for binary classification
model.add(Dense(1, activation='sigmoid'))

# Compile the model
# SGD optimizer with learning rate of 0.002 and momentum of 0.8
model.compile(optimizer=SGD(lr=0.002, momentum=0.8), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Model summary
model.summary()

In [ ]:
# Test loading an image
path = './data/train/bee'
name = '22874935_066a5b774c_m.jpg'
fullname = f'{path}/{name}'
load_img(fullname)

In [ ]:
img = load_img(fullname, target_size=(150, 150))
x = np.array(img)
x.shape

In [ ]:
# Load train data
train_gen = ImageDataGenerator(rescale=1./255)
train_ds = train_gen.flow_from_directory(
    './data/train',
    target_size=(150, 150), 
    batch_size=20,
    shuffle=True,
    class_mode='binary'
)

In [ ]:
train_ds.class_indices

In [ ]:
X, y = next(train_ds)

In [ ]:
y[:5]

In [ ]:
# Load test data
val_gen = ImageDataGenerator(rescale=1./255)
val_ds = val_gen.flow_from_directory(
    './data/test',
    target_size=(150, 150), 
    batch_size=20,
    shuffle=True,
    class_mode='binary'
)

In [ ]:
# Train model
history = model.fit(train_ds, epochs=10, validation_data=val_ds)